In [ ]:
! pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!unzip archive

unzip:  cannot find or open archive, archive.zip or archive.ZIP.


In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import pandas as pd
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

In [8]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [11]:
df.shape

(1600000, 6)

In [14]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [16]:
twitter_data.shape

(1600000, 6)

In [17]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [19]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [21]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [22]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [23]:
port_stem = PorterStemmer()

In [26]:
def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [27]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [28]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [29]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [30]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [31]:
print(Y)

[0 0 0 ... 1 1 1]


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=99)

In [57]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [58]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9452620 stored elements and shape (1280000, 460945)>
  Coords	Values
  (0, 346912)	0.9014265291092924
  (0, 12373)	0.43293211086494143
  (1, 51205)	0.8327287470675659
  (1, 453019)	0.3516648556728611
  (1, 384030)	0.2962380709983827
  (1, 420394)	0.30844394690611904
  (2, 205860)	0.3813689832271099
  (2, 154756)	0.24888513119289019
  (2, 104791)	0.4637531646523823
  (2, 22520)	0.5422205891466647
  (2, 443201)	0.28405322302501806
  (2, 87894)	0.27463043061333375
  (2, 441473)	0.35698125742676823
  (3, 90980)	0.644165161953797
  (3, 4426)	0.35492302255654273
  (3, 456037)	0.4136669469455318
  (3, 94650)	0.3156496872969802
  (3, 359897)	0.34940778855750104
  (3, 249646)	0.25737136863645926
  (4, 258943)	0.5751725291836107
  (4, 152048)	0.6301076851170194
  (4, 66457)	0.279481754249674
  (4, 239441)	0.44048929143683446
  (5, 94650)	0.433173775143978
  (5, 286174)	0.14259268050677248
  :	:
  (1279994, 132493)	0.26754767714677596


In [59]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289076 stored elements and shape (320000, 460945)>
  Coords	Values
  (0, 12748)	0.20539149875277105
  (0, 149441)	0.12643433947576532
  (0, 150332)	0.18844936932650042
  (0, 157093)	0.17871098445614664
  (0, 177554)	0.3242854218727004
  (0, 227573)	0.2155894417787674
  (0, 266104)	0.24157315667940824
  (0, 267284)	0.15674476607177648
  (0, 273937)	0.47261215900135545
  (0, 279081)	0.2587933751689233
  (0, 287817)	0.19876228839898077
  (0, 334151)	0.20740971565759486
  (0, 361115)	0.2669052979018404
  (0, 365352)	0.22755414480173647
  (0, 414458)	0.23845839118927187
  (0, 423684)	0.2950734193601413
  (1, 34947)	0.34755282516246916
  (1, 129833)	0.3759560468089153
  (1, 138527)	0.24107216831134193
  (1, 168357)	0.45926987981224493
  (1, 234313)	0.3345176610009746
  (1, 267284)	0.2037655190269352
  (1, 273239)	0.29721439259694776
  (1, 301411)	0.2049048830209043
  (1, 308607)	0.2809187747854183
  :	:
  (319995, 128975)	0.32901

In [60]:
model = LogisticRegression(max_iter=1000)

In [61]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [62]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [63]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.79143515625


In [64]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(y_test, X_test_prediction)

In [65]:
print('Accuracy score on the testing data :', testing_data_accuracy)

Accuracy score on the testing data : 0.774728125


In [66]:
import pickle

filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [67]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [70]:
X_new = X_test[900]
print(y_test[900])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[1]
Positive Tweet
